In [ ]:
from ratdata import data_manager as dm, process, ingest
import pathlib

In [ ]:
dm.db_connect('rat_data.db')

In [ ]:
recording_files = dm.RecordingFile.select()

In [ ]:
recalculate = True
scale = 1

for i, f in enumerate(recording_files):
    if recalculate or (f.slice.count() == 1 and f.slice.get().updated):
        fname = pathlib.Path(f.dirname) / f.filename
        recording = ingest.read_mce_matlab_file(str(fname))
        if f.slice.count() == 1:
            slice = f.slice.get()
            data, tt = process.trim_recording(recording.electrode_data.mean(0),
                                            int(1/recording.dt), slice.start, slice.length)
        else:
            data = recording.electrode_data.mean(0)
        recording_beta_power = process.compute_power_in_frequency_band(data, 13, 30, int(1/recording.dt))
        recording_total_power = process.compute_power_in_frequency_band(data, 10, 100, int(1/recording.dt))
        beta_power_no_oof = process.power_in_band_no_oof(data, 13, 30, int(1/recording.dt), 2, 100, scale)
        rp_id = dm.upsert_power_record(f, recording_beta_power, recording_total_power, beta_power_no_oof)
        print('[%d/%d] %s: %d' % (i + 1, len(recording_files), f.filename, rp_id))
    